# Get data
```
mkdir -p /home/paul/cmet/data/20200811-maclaren-fs-7.1-samseg-and-aseg/
cd /home/paul/cmet/data/20200811-maclaren-fs-7.1-samseg-and-aseg/
aws s3 cp s3://cmet-scratch/maclaren-cmeds/demographics.tsv .
aws s3 cp \
  --recursive \
  --exclude "*" \
  --include "*.stats" \
  s3://cmet-scratch/20200811-maclaren-fs-7.1-samseg-and-aseg/ .
```

The following runs did not recon (no aseg.stats):
- sub-01_run-21
- sub-01_run-27
- sub-01_run-31
- sub-01_run-34
- sub-01_run-36
- sub-01_run-37

In [1]:
import json
import os
import fnmatch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# imports find_json_files(); load_json_file(); load_dataset(); load_fs_dataset()
from cmeds import *
# imports calc_cvs(); session_permute(); monte_carlo_perm_test
from test_retest import *

In [2]:
structs_of_interest = [
    'Left-Lateral-Ventricle',
    'Left-Hippocampus',
    'Left-Amygdala',
    'Left-Caudate',
    'Left-Putamen',
    'Right-Lateral-Ventricle',
    'Right-Hippocampus',
    'Right-Amygdala',
    'Right-Caudate',
    'Right-Putamen'
]
drop_subjects = []
demofile = '/home/paul/cmet/data/20200811-maclaren-fs-7.1-samseg-and-aseg/demographics.tsv'
datadir = '/home/paul/cmet/data/20200811-maclaren-fs-7.1-samseg-and-aseg/'
maclaren_fs_vol_df = load_fs_dataset(datadir, demofile, structs_of_interest, drop_subjects=[])

# Add some lateral regions together so we can directly compare to table 1 in MacLaren et al.
regions = [             
            [ ['Left-Hippocampus', 'Right-Hippocampus'],'Hippocampus' ],
            [ ['Left-Lateral-Ventricle', 'Right-Lateral-Ventricle'],'Lateral-Ventricles' ],
            [ ['Left-Amygdala', 'Right-Amygdala',],'Amygdala' ],
            [ ['Left-Putamen', 'Right-Putamen'],'Putamen' ],
            [ ['Left-Caudate', 'Right-Caudate'],'Caudate' ],
          ]
maclaren_fs_vol_df = add_regions(maclaren_fs_vol_df,regions)

Dropping the following subjects []


In [13]:
# The column name that holds session info in the demographics.tsv
session_col='session'
# The column name that holds subject info in the demographics.tsv
subject_col='subject_num'
structs_of_interest = ['Hippocampus', 'Lateral-Ventricles', 'Amygdala', 'Putamen', 'Caudate']


#session_list= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
# Removing the following sessions due to sub1 processing errors:
# 11 14 16 17 18 19
session_list= [1,2,3,4,5,6,7,8,9,10,12,13,15,20]
subject_list= [1,2,3]

#cvs_macmethod = calc_cvs(maclaren_fs_vol_df,subject_list,session_list,subject_col,session_col,structs_of_interest,method='maclaren')

n = 1000
perm_test_reults = monte_carlo_perm_test(maclaren_fs_vol_df, subject_list, session_list, subject_col, session_col, structs_of_interest, n_itrs=n, method='gluer')

Comparable to [Table 1 in MacLaren et al](https://www.nature.com/articles/sdata201437/tables/2)

In [14]:
perm_test_reults

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,8589.077381,12267.938095,3563.077381,9295.917857,7061.282143
total-cov,24.318034,24.382454,24.440787,24.353302,24.326318
session-cov,1.937433,0.879134,3.389617,1.982030,1.521334
abs-diff-cov,22.380600,23.503320,21.051170,22.371272,22.804983
p-vals,0.001000,0.000000,0.001000,0.000000,0.000000
